In [27]:
import json

def get_tickets():
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data

def get_ticket(index):
    with open('support_tickets.json', 'r') as file:
        data = json.load(file)
    return data['tickets'][index]

def stringify_ticket(ticket):
    return f"Title: {ticket['title']}\nDescription: {ticket['description']}"

ticket = get_ticket(0)
print(stringify_ticket(ticket))


Title: Auto Assignment Not Working
Description: I haven't received any auto assignments for the past 2 hours. I've checked my status and I'm set as 'Available'. My queue is empty but I know there are cases in the general pool. Please help troubleshoot why I'm not getting assignments.


In [33]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
import markdown

def load_knowledge_base():
    with open('knowledge_wiki.md', 'r') as file:
        kb_content = file.read()
    
    kb_text = markdown.markdown(kb_content)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len,
    )
    
    chunks = text_splitter.split_text(kb_text)
    embeddings = OllamaEmbeddings(model="llama3.2:1b")
    vectorstore = FAISS.from_texts(chunks, embeddings)
    return vectorstore


def find_relevant_knowledge(ticket_string, top_k=3):
    vectorstore = load_knowledge_base()
    relevant_chunks = vectorstore.similarity_search(ticket_string, k=top_k)
    return [doc.page_content for doc in relevant_chunks]

In [34]:
ticket = get_ticket(0)
ticket_string = stringify_ticket(ticket)
relevant_knowledge = find_relevant_knowledge(ticket_string)
print("\nRelevant knowledge base sections:")
for i, chunk in enumerate(relevant_knowledge, 1):
    print(f"\nChunk {i}:")
    print(chunk)


Relevant knowledge base sections:

Chunk 1:
<h3><strong>Troubleshooting Auto Assignment Issues</strong></h3>
<p>If you are not receiving auto assignments despite being marked as "Available," follow these steps:
1. <strong>Verify System Status:</strong> Check for ongoing outages or maintenance in the assignment system.
2. <strong>Check Queue Configuration:</strong> Ensure your queue is properly configured and there are no conflicting filters.
3. <strong>Manually Refresh Status:</strong> Toggle your availability off and on again.
4. <strong>Contact Support:</strong> If the issue persists, reach out to the system administrator for further investigation.</p>
<hr />
<h3><strong>Blocking Cases from Specific Regions</strong></h3>
<p>If you need to restrict cases from a particular region:
1. <strong>Access Your Assignment Rules:</strong> Navigate to your settings and find the "Case Assignment Preferences" section.
2. <strong>Modify Region Filters:</strong> Select the regions you wish to exclu

In [6]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2:1b")

response = llm.invoke("What is the capital of France?")
print(response)


The capital of France is Paris.


In [8]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

chat_model = ChatOllama(model="llama3.2:1b")

messages = [
    SystemMessage(content="You are a helpful AI assistant."),
    HumanMessage(content="Tell me a short joke about programming.")
]

response = chat_model.invoke(messages)
print(response.content)

A programmer walked into a library and asked the librarian, "Do you have any books on Pavlov's dogs and Schrödinger's cat?" The librarian replied, "It rings a bell, but I'm not sure if it's here or not."
